In [63]:
import sys
import csv
from selenium import webdriver
import time

# default path to file to store data
path_to_file = "../datasets/scraped_final_list_URL_TripAdvisor.csv"

# default number of scraped pages
num_page = 1000

# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com/Restaurant_Review-g55711-d649839-Reviews-Pappas_Bros_Steakhouse-Dallas_Texas.html"
# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

PATH = "/usr/bin/chromedriver"
# Import the webdriver
driver = webdriver.Chrome(PATH)
driver.get(url)

In [65]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Open the file to save the review
# csvFile = open(path_to_file, 'a', encoding="utf-8")
# csvWriter = csv.writer(csvFile)

def expand_reviews(locator, base_element = driver):
    element = WebDriverWait(base_element, timeout=10).until(
            EC.presence_of_element_located(locator) 
    )
    if element is not None:
        element.click() 

reviews_div = driver.find_element_by_id("REVIEWS")
num_pages = int(reviews_div.find_elements_by_class_name("pageNum.last")[0].text)

print(f"Num pages are {num_pages}")
# change the value inside the range to save more or less reviews
for i in range(0, 1):
    
#     # expand the review 
    more_span = "//span[@class='taLnk ulBlueLinks']"
    expand_locator = (By.XPATH, more_span)
        
    try:
        expand_reviews(expand_locator, base_element = reviews_div)
    except Exception as e:
        expand_reviews(expand_locator)
    
    # Wait for the expansion to take place    
    time.sleep(2)
    
    # Make sure that all the reviews are loaded
    max_num_tries = 5
    num_tries = 0
    num_reviews_per_page = 10
    
    container = driver.find_elements_by_xpath(".//div[@class='review-container']")
    while len(container) != num_reviews_per_page and num_tries < max_num_tries:
        container = driver.find_elements_by_xpath(".//div[@class='review-container']")
        num_tries+=1
    
    print(f"Num reviews in this page: {len(container)}")
#     print(container)
    for j in range(len(container)):
        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        try:
            review = container[j].find_element_by_xpath(".//span[@class='postSnippet']").text.replace("\n", " ")
        except:
            review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")
        
        print([date, rating, title, review]) 

    # change the page
    driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()

driver.close()


KeyboardInterrupt: Interrupted by user